In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import preprocessing

C:\Users\yangt\Anaconda3\Ana\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
seed = 7
np.random.seed(seed)

In [3]:
train_data = pd.read_csv('cleanTrain.csv')
test_data = pd.read_csv('cleanToPredict.csv')

X_1 = train_data.select_dtypes(include=[object])
X_1.head()



# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = X_1.apply(le.fit_transform)
X_2.head()


train_data['CREDIT Grade'] = X_2['CREDIT Grade']
train_data['Loan Purpose'] = X_2['Loan Purpose']
train_data['State'] = X_2['State']
train_data['Home Ownership'] = X_2['Home Ownership']
train_data['FICO Range'] = X_2['FICO Range']
train_data['Employment Length'] = X_2['Employment Length']
train_data['Education'] = X_2['Education']

Y_1 = test_data.select_dtypes(include=[object])
Y_1.head()



# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
Y_2 = Y_1.apply(le.fit_transform)
Y_2.head()

store = test_data['Loan ID'].values
del test_data['Loan ID']
test_data['CREDIT Grade'] = Y_2['CREDIT Grade']
test_data['Loan Purpose'] = Y_2['Loan Purpose']
test_data['State'] = Y_2['State']
test_data['Home Ownership'] = Y_2['Home Ownership']
test_data['FICO Range'] = Y_2['FICO Range']
test_data['Employment Length'] = Y_2['Employment Length']
test_data['Education'] = Y_2['Education']

In [15]:

X = train_data.values[:,0:-1]

Y = train_data.values[:,-1]

#Testing!!
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)
print(X_train.shape)
print(y_train.shape)

import tensorflow as tf

model = tf.keras.Sequential()
lossType = tf.keras.losses.binary_crossentropy
opt = tf.train.AdamOptimizer()

i_shape = (26,)
model.add(tf.keras.layers.Dense(60, input_shape = i_shape, kernel_initializer = 'normal', activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1,activation = tf.nn.sigmoid))
model.compile(optimizer = opt, loss = lossType)
model.fit(X_train,y_train,batch_size = 32, epochs = 5)
    

preds = model.predict(X_test)

print("Accuracy is", accuracy_score(y_test,preds)*100)

(13984, 26)
(13984,)
Epoch 1/5
13984/13984 [==============================] - 1s 57us/step - loss: 7.9338
Epoch 2/5
13984/13984 [==============================] - 1s 41us/step - loss: 7.9357
Epoch 3/5
13984/13984 [==============================] - 0s 31us/step - loss: 7.9357
Epoch 4/5
13984/13984 [==============================] - 0s 31us/step - loss: 7.9357
Epoch 5/5
13984/13984 [==============================] - 0s 32us/step - loss: 7.9357
Accuracy is 50.70070070070069


In [4]:
#gini
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=1200,min_samples_leaf=44)
clf_gini.fit(X_train, y_train)

y_pred = clf_gini.predict(X_test)

In [5]:
#accuracy score
print ("Accuracy is ", accuracy_score(y_test,y_pred)*100)

Accuracy is  80.7307307307


In [6]:
#accuracy score entropy

clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=1200, min_samples_leaf=44)
clf_entropy.fit(X_train, y_train)

y_pred_entro = clf_entropy.predict(X_test)

print("Accuracy is", accuracy_score(y_test,y_pred_entro)*100)

Accuracy is 80.6973640307


In [7]:
from sklearn.ensemble import AdaBoostClassifier


clf_ada = AdaBoostClassifier(base_estimator = clf_entropy,n_estimators = 100, learning_rate = 0.01,random_state = 100)
clf_ada.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1200,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=44,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best'),
          learning_rate=0.01, n_estimators=100, random_state=100)

In [8]:
y_pred_ada = clf_ada.predict(X_test)
print("Accuracy is", accuracy_score(y_test,y_pred_ada)*100)


Accuracy is 83.4834834835


In [9]:
#test_data.isnull().sum()
#test_data.head()

ok = test_data.values[:,1:-1]
print(ok)
y_pred_1 = clf_gini.predict(ok)
y_pred_randomForest = clf_random.predict(ok)
y_pred_ada_test = clf_ada.predict(ok)
#print(y_pred_1)


[[10000.0 0.1016 36 ..., 86.0 2 4]
 [4800.0 0.1361 36 ..., 19.0 2 10]
 [10000.0 0.1212 36 ..., 23.0 2 10]
 ..., 
 [8484.93 0.1411 36 ..., 46.0 3 3]
 [4595.73 0.1292 36 ..., 0.0 1 1]
 [25000.0 0.1114 36 ..., 9.0 2 6]]


ValueError: Number of features of the model must match the input. Model n_features is 26 and input n_features is 25 

In [ ]:
#df_pred = pd.DataFrame(store,y_pred_1)
df_pred = pd.DataFrame({'$a':df1['Loan ID', '$b': y_pred_ada_test})
#df_pred = pd.Panel({'Loan ID': store, 'Status (Fully Paid=1, Not Paid=0)': y_pred_1})


In [ ]:
df_pred.columns=['Loan ID','Status (Fully Paid=1, Not Paid=0)']
df_pred.head()

In [ ]:
df_pred.to_csv('hi.csv', encoding='utf-8',index= False)